# Initial Setup
> This notebook was created with Weaviate `1.25` and the Weaviate Client `4.6`

## Prepare API keys

If you are new to Google AI Studio, register at [https://aistudio.google.com](https://aistudio.google.com) and head to [https://aistudio.google.com/app/u/0/apikey](https://aistudio.google.com/app/u/0/apikey) to create your API key.

And set them inside the [.env](../.env) file

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")
print(AI_STUDIO_KEY)

## Deploy Weaviate

Weaviate offers 3 deployment options:
* Embedded
* Self-hosted - with Docker Compose
* Cloud deployment - [Weaviate Cloud Service](https://console.weaviate.cloud/)

# Time to Build
## Connect to Weaviate

In [ ]:
import weaviate, json

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
       "X-Google-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

In [ ]:
# import weaviate, os, json

# Connect with Weaviate Embedded
# client = weaviate.connect_to_embedded(
#     version="1.24.14",
#     headers={
#         "X-Google-Api-Key": AI_STUDIO_KEY,
#     })

# # Connect to a Weaviate Cloud instance
# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_ADMIN")),

#     headers={
#        "X-Google-Api-Key": AI_STUDIO_KEY,
#     }
# )

# client.is_ready()

## Create a collection
[Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/configuration/schema-configuration)

In [ ]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection here - with Google as a vectorizer
client.collections.create(
    name="Jeopardy",
    vectorizer_config=Configure.Vectorizer.text2vec_palm(
        model_id="embedding-001",

        project_id="no need for this", # this needs fixing
        api_endpoint="generativelanguage.googleapis.com"
    ),
)

## Import data

### Sample Data

In [ ]:
import json

with open("./jeopardy-100.json") as file:
    data_100 = json.load(file)

print(json.dumps(data_100[0], indent=2))

### Insert Many
[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [ ]:
# Insert data
jeopardy = client.collections.get("Jeopardy")
jeopardy.data.insert_many(data_100)

## Check the object count
[Weaviate Docs - Aggregate](https://weaviate.io/developers/weaviate/search/aggregate#retrieve-the-count-meta-property)

In [ ]:
questions = client.collections.get("Jeopardy")
questions.aggregate.over_all()

### Data preview

In [ ]:
# Show data preview
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(limit=4)

for item in response.objects:
    # print(item.uuid, json.dumps(item.properties, indent=2))
    print(item.uuid, item.properties)

In [ ]:
# Show data preview - with vectors
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

## Close the client when done

In [ ]:
client.close()